In [36]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'
os.environ['TF_NUM_INTRAOP_THREADS'] = '0'
os.environ['TF_NUM_INTEROP_THREADS'] = '0'
os.environ['TF_KERAS'] = '1'
os.environ['TF_MKL'] = '1'

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


In [37]:
# Load the dataset
data = pd.read_csv('review.csv')

In [38]:
data.head()

,sentiment,review
0,1.0,Good but need updates and improvements
1,0.0,"Worst mobile i have bought ever, Battery is dr..."
2,1.0,when I will get my 10% cash back.... its alrea...
3,1.0,Good
4,0.0,The worst phone everThey have changed the last...


In [39]:
data.tail()

,sentiment,review
14670,1.0,"I really like the phone, Everything is working..."
14671,1.0,The Lenovo K8 Note is awesome. It takes best p...
14672,1.0,Awesome Gaget.. @ this price
14673,1.0,This phone is nice processing will be successf...
14674,1.0,Good product but the pakeging was not enough.


In [40]:
# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['sentiment'], test_size=0.001)

In [41]:
# Tokenize the text and pad the sequences
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=100, padding='post', truncating='post')
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=100, padding='post', truncating='post')

In [42]:
# Define the model
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=100),
    tf.keras.layers.GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

In [43]:
# Compile the model
opt = tf.compat.v1.train.AdamOptimizer()
opt = tf.compat.v1.train.experimental.enable_mixed_precision_graph_rewrite(opt)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [44]:
# Train the model
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), use_multiprocessing=True)

Epoch 1/5
  4/459 [..............................] - ETA: 8s - loss: 0.6935 - accuracy: 0.4922  

2023-04-16 03:10:10.797243: W tensorflow/core/grappler/optimizers/auto_mixed_precision.cc:2303] No (suitable) GPUs detected, skipping auto_mixed_precision graph optimizer


459/459 [==============================] - 9s 18ms/step - loss: 0.5197 - accuracy: 0.7508 - val_loss: 0.2907 - val_accuracy: 0.8667
Epoch 2/5
  7/459 [..............................] - ETA: 8s - loss: 0.3772 - accuracy: 0.8616

2023-04-16 03:10:19.333768: W tensorflow/core/grappler/optimizers/auto_mixed_precision.cc:2303] No (suitable) GPUs detected, skipping auto_mixed_precision graph optimizer


459/459 [==============================] - 8s 18ms/step - loss: 0.3263 - accuracy: 0.8723 - val_loss: 0.5038 - val_accuracy: 0.8000
Epoch 3/5
459/459 [==============================] - 8s 18ms/step - loss: 0.2812 - accuracy: 0.8930 - val_loss: 0.4648 - val_accuracy: 0.8667
Epoch 4/5
459/459 [==============================] - 8s 18ms/step - loss: 0.2583 - accuracy: 0.9048 - val_loss: 0.4960 - val_accuracy: 0.8667
Epoch 5/5
459/459 [==============================] - 8s 18ms/step - loss: 0.2386 - accuracy: 0.9134 - val_loss: 0.4540 - val_accuracy: 0.8667


In [69]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 22ms/step - loss: 0.4540 - accuracy: 0.8667


In [70]:
# Print the model's accuracy
print('Test accuracy:', accuracy)

Test accuracy: 0.8666666746139526


In [71]:
# Get review from user input
user_review = input("Enter your review: ")

# Preprocess user review
user_review = tokenizer.texts_to_sequences([user_review])
user_review = pad_sequences(user_review, maxlen=100, padding='post', truncating='post')

Enter your review: Bad item


In [72]:
# Predict sentiment for user review
sentiment = model.predict(user_review)

# Print the predicted sentiment
print(sentiment)
if sentiment > 0.6:
    print("Positive")
elif sentiment < 0.6 and sentiment >= 0.1:
    print("Neutral")
else:
    print("Negative")

1/1 [==============================] - 0s 17ms/step
[[0.05548866]]
Negative


# 

In [59]:
from tensorflow.keras.models import model_from_json

In [60]:
model_json = model.to_json()

In [61]:
with open('model.json', "w") as json_file:
    json_file.write(model_json)

In [62]:
model.save_weights("model.h5")

In [ ]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

In [ ]:
# Get review from user input
user_review = input("Enter your review: ")

# Preprocess user review
user_review = tokenizer.texts_to_sequences([user_review])
user_review = pad_sequences(user_review, maxlen=100, padding='post', truncating='post')

In [ ]:
# Predict sentiment for user review
sentiment = loaded_model.predict(user_review)

# Print the predicted sentiment
print(sentiment)
if sentiment > 0.6:
    print("Positive")
elif sentiment < 0.6 and sentiment >= 0.1:
    print("Neutral")
else:
    print("Negative")